# Create imcrop and dlcrop from h&e image (imsrc), roi image (roi mask 1-12), and

In [2]:
OPENSLIDE_PATH = r'C:\Users\Kevin\Desktop\openslide-win64-20221217\bin'

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [ ]:
# import pandas as pd
import numpy as np
import os
from PIL import Image
import os
import cv2
Image.MAX_IMAGE_PIXELS = None
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import scipy.stats as stats
import torchvision.transforms as transforms
from glob import glob
from time import time
from skimage.measure import label
import math
from skimage.morphology import closing, square, remove_small_objects, remove_small_holes
from math import atan2, degrees
from copy import deepcopy
from time import time
from natsort import natsorted

In [ ]:
def rotate_image_cv2(mat, angle):
    """
    Helper function used in pipeline:
    Rotates an image (angle in degrees) and expands image to avoid cropping
    Image has to be uint8
    """

    height, width = mat.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0,0])
    abs_sin = abs(rotation_mat[0,1])

    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h),	flags = cv2.INTER_NEAREST)
    return rotated_mat

In [ ]:
imsrc = r'\\shelter\Kyu\skin_aging\clue_cohort\HE1um'
dlsrc = r'\\shelter\Kyu\skin_aging\clue_cohort\DLmask1um'
roisrc = r'\\shelter\Kyu\skin_aging\clue_cohort\annotations\roi\labeledmask_20rsf'

In [ ]:
roiflag = True

minTA = 20000
minTAhole = 100
minDermhole = 5000
minepisize = 1000
whitespace = 12

if roiflag:
    imdst = os.path.join(imsrc, 'imcrop_roiv2')
    if not os.path.exists(imdst): os.mkdir(imdst)
    imdst2 = os.path.join(imdst, 'imcrop_roi_10umv2')
    if not os.path.exists(imdst2): os.mkdir(imdst2)
    dldst = os.path.join(dlsrc, 'dlcrop_roiv2')
    if not os.path.exists(dldst): os.mkdir(dldst)
else:
    imdst = os.path.join(imsrc, 'imcropv2')
    if not os.path.exists(imdst): os.mkdir(imdst)
    imdst2 = os.path.join(imdst, 'imcrop_10umv2')
    if not os.path.exists(imdst2): os.mkdir(imdst2)
    dldst = os.path.join(dlsrc, 'dlcropv2')
    if not os.path.exists(dldst): os.mkdir(dldst)

imlist = [os.path.join(imsrc, _) for _ in os.listdir(imsrc) if _.endswith('tif')]
dllist = [os.path.join(dlsrc, _) for _ in os.listdir(dlsrc) if _.endswith('tif')]
imlist = natsorted(imlist)
dllist = natsorted(dllist)

df = []

idx=0
imname = imlist[0]
dlname = dllist[0]
start = time()


imfn, ext = os.path.splitext(os.path.basename(imname))
dlfn, ext = os.path.splitext(os.path.basename(dlname))
roiname = os.path.join(roisrc, imfn+'.png')

isim = os.path.exists(os.path.join(imdst, '{}_sec{:02d}.png'.format(imfn, 1)))
isdl = os.path.exists(os.path.join(imdst, '{}_sec{:02d}.png'.format(dlfn, 1)))
if isim and isdl:
    print('continue')

print(imfn)
print('idx:',idx)

# open image
dl = Image.open(dlname)
im = Image.open(imname)

if not im.size == dl.size:
    raise Exception("im and dl dimensions do not match")

TAbig = np.array(dl)

if roiflag:
    roi = np.array(Image.open(roiname))

# downsize to expedite (downsize deeplab mask)
(width, height) = (dl.width // 10, dl.height // 10)
# dl_resized = dl.resize((width, height), resample=0)
# TA = np.array(dl_resized)
# print(TA.shape)
TA = cv2.resize(TAbig, dsize=(width,height), interpolation=cv2.INTER_NEAREST)
# print(TA.shape)
# raise
sure_fg = closing((2 < TA) & (TA < whitespace - 1), square(3))  # 13sec # close small holes in DLmask
sure_fg = remove_small_objects(sure_fg, min_size=minTA, connectivity=2)  # 6sec
sure_fg = remove_small_holes(sure_fg, area_threshold=minTAhole).astype(np.uint8)  # 7sec
# define background
bw = closing(TA < whitespace, square(3))  # 12 is background
bw = remove_small_objects(bw, min_size=minTA, connectivity=2)
bw = remove_small_holes(bw, area_threshold=minTAhole)
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(bw.astype(np.uint8), cv2.MORPH_OPEN, kernel, iterations=2)  # 2sec #difference between erosion and dilation, gives outline of object
sure_bg = cv2.dilate(opening, kernel, iterations=3)

In [ ]:
plt.imshow(sure_fg)

In [ ]:
# define middleground
unknown = cv2.subtract(sure_bg, sure_fg).astype('bool') # foreground - background = middleground
# label that background is 1 and objects are 2~N and middleground is zero
sure_fg_label = label(sure_fg).astype(np.int32)
sure_fg_label = sure_fg_label + 1
sure_fg_label[unknown] = 0
# perform watershed based on the marker
TAbgr = cv2.cvtColor(TA, cv2.COLOR_GRAY2BGR)
label_image = cv2.watershed(TAbgr, sure_fg_label)
# iterate each section
epi = (TA == 1) | (TA == 2)
derm = (2 < TA) & (TA < whitespace)
derm = remove_small_holes(derm, area_threshold=minDermhole)
epi2 = epi & ~derm
epi2 = remove_small_objects(epi2, min_size=minepisize, connectivity=2)
numsecmax = np.max(label_image)
print(round(time() - start), 'sec elapsed for overhead')